# Daily Greeting - Prompt Iteration

This notebook lets you iterate on prompting techniques with cached weather and literature data.

In [1]:
import json
import re
import random
import logging
from daily_greeting import send_ollama_request, send_ollama_image_request
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

## Fetch Weather Data

In [2]:
# Fetch fresh data
from daily_greeting import get_weather_data
weather = get_weather_data()

logging.info("Weather:")
logging.info(json.dumps(weather, indent=2))

INFO: Fetching forecast url for coordinates: 42.2688, -71.8088
INFO: Fetching forecast data...
INFO: Weather:
INFO: {
  "overnight": {
    "dayOfWeek": "Tonight",
    "precipitation": 0,
    "description": "Clear, with a low around 51. West wind around 9 mph."
  },
  "sunrise": {
    "temperature": 51,
    "humidity": 93,
    "dewpoint": 9.444444444444445,
    "windSpeed": "8 mph",
    "windDirection": "W",
    "precipitation": 0,
    "conditions": "Sunny"
  },
  "today": {
    "dayOfWeek": "Saturday",
    "precipitation": 2,
    "description": "Sunny, with a high near 79. West wind around 8 mph."
  }
}


# Fetch Literature Data

Run it through the LLM up to some number of times to see whether it thinks it is ineteresting enough to use.

In [ ]:
from daily_greeting import get_random_literature, format_literature

attempts = 0
logging.info("Starting literature selection process.")
while attempts < 5:
    literature = get_random_literature()
    if not literature:
        logging.warning("No literature found, trying again...")
        attempts += 1
        continue

    formatted_lit = format_literature(literature)
    literature_prompt = f"""Please evaluate whether the following literary excerpt is suitable material from which to source themes, tone, imagery, or metaphor.

{formatted_lit}

Respond in this exact format strictly:
REASONING: One sentence reasoning about the suitability of the text.
VERDICT: YES if suitable NO if not"""

    print("\n" + "="*50)
    print("LITERATURE EVALUATION PROMPT")
    print("="*50)
    print(literature_prompt)

    evaluation = send_ollama_request(literature_prompt.format())

    print("\n" + "="*50)
    print("LITERATURE EVALUATION")
    print("="*50)
    print(evaluation)

    attempts += 1
    if "VERDICT: YES" in evaluation.upper():
        break

if attempts == 5:
    logging.error("Failed to find suitable literature after 5 attempts.")
else:
    logging.info(f"Suitable literature found in {attempts} attempts.")

logging.info("Literature:")
logging.info(json.dumps(literature, indent=2))

INFO: Starting literature selection process.
INFO: Fetching books from Gutendex page 5...
INFO: Found 32 books on page 5.
INFO: Fetching book content of book...
INFO: Sending request to Ollama model llama3.2:3b...



LITERATURE EVALUATION PROMPT
Please evaluate whether the following literary excerpt is suitable material from which to source themes, tone, imagery, or metaphor.

Literature excerpt:
"The Twilight of the Idols; or, How to Philosophize with the Hammer. The Antichrist: Complete Works, Volume Sixteen" by Nietzsche, Friedrich Wilhelm (1844-1900):
of
the _nervus sympathies_—stimulate our instinct of causality: we will
have a reason which will account for our feeling thus or thus,—for
feeling ill or well. We are never satisfied by merely ascertaining
the fact that we feel thus or thus: we admit this fact—we become
conscious of it—only when we have attributed it to some kind of
motivation. Memory, which, in such circumstances unconsciously becomes
active, adduces former conditions of a like kind, together with the
causal interpretations with which they are associated,—but not their
real cause. The belief that the ideas, the accompanying processes
of consciousness, have been the causes, is ce

INFO: Suitable literature found in 1 attempts.
INFO: Literature:
INFO: {
  "title": "The Twilight of the Idols; or, How to Philosophize with the Hammer. The Antichrist: Complete Works, Volume Sixteen",
  "author": {
    "name": "Nietzsche, Friedrich Wilhelm",
    "birth_year": 1844,
    "death_year": 1900
  },
  "excerpt": "of\r\nthe _nervus sympathies_\u2014stimulate our instinct of causality: we will\r\nhave a reason which will account for our feeling thus or thus,\u2014for\r\nfeeling ill or well. We are never satisfied by merely ascertaining\r\nthe fact that we feel thus or thus: we admit this fact\u2014we become\r\nconscious of it\u2014only when we have attributed it to some kind of\r\nmotivation. Memory, which, in such circumstances unconsciously becomes\r\nactive, adduces former conditions of a like kind, together with the\r\ncausal interpretations with which they are associated,\u2014but not their\r\nreal cause. The belief that the ideas, the accompanying processes\r\nof conscio


LITERATURE EVALUATION
REASONING: The excerpt's focus on the human tendency to attribute causes to feelings and memories, along with its philosophical tone, suggests that it could be a rich source for exploring themes related to causality, motivation, and the limitations of psychological explanation.

VERDICT: YES


# Fetch Navidrome Album

Request five random albums from the Navidrome server. Select one which will pair best with the selected literature.

In [ ]:
from daily_greeting import get_navidrome_albums, format_albums

albums = get_navidrome_albums(count=5)
formatted_albums = format_albums(albums)

album_prompt = f"""Please select one of the following albums which would pair most interestingly with the selected literary excerpt, whether by contrast or by complement.

{formatted_albums}

{formatted_lit}

Respond in this exact format strictly:
REASONING: Two or three sentences reasoning about the best album choice.
VERDICT: 1 or 2 or 3 or 4 or 5 (the number of the selected album from the list above) and nothing else"""

print("\n" + "="*50)
print("ALBUM SELECTION PROMPT")
print("="*50)
print(album_prompt)

evaluation = send_ollama_request(album_prompt, keep_alive=0)

print("\n" + "="*50)
print("ALBUM SELECTION")
print("="*50)
print(evaluation)

# extract the selected album number using regex
match = re.search(r'VERDICT:\s*(\d+)', evaluation)
if match:
    selection = int(match.group(1)) - 1
    # check if selection is within range
    if selection < 0 or selection >= len(albums):
        logging.error(f"Album selection {selection + 1} is out of range.")
        selection = random.randint(0, 4)  # fallback to a random selection
    else:
        logging.info(f"Selected album #{selection + 1}: {albums[selection]['name']} by {albums[selection]['artist']}")
else:
    # handle parsing failure
    logging.error("Could not parse album selection from response")
    selection = random.randint(0, 4)  # Fallback to a random selection

album = albums[selection]

INFO: Fetching random albums from Navidrome server...
INFO: Sending request to Ollama model llama3.2:3b...



ALBUM SELECTION PROMPT
Please select one of the following albums which would pair most interestingly with the selected literary excerpt, whether by contrast or by complement.

Albums:
[1] "It's You Or No One" by Sarah Vaughan (1959) - Genres: Jazz
[2] "Beginning Rock Guitar for Kids" by Jimmy Brown (1994) - Genres: Rock
[3] "Schubert: Piano Trio Nr. 2, D.929: Trio-Sonate, D. 28" by Beaux Arts Trio (1993) - Genres: Chamber Music
[4] "Yessongs (Disc 2)" by Yes (1973) - Genres: Progressive Rock
[5] "Simple Things" by Zero 7 (Unknown) - Genres: Unknown genre

Literature excerpt:
"The Twilight of the Idols; or, How to Philosophize with the Hammer. The Antichrist: Complete Works, Volume Sixteen" by Nietzsche, Friedrich Wilhelm (1844-1900):
of
the _nervus sympathies_—stimulate our instinct of causality: we will
have a reason which will account for our feeling thus or thus,—for
feeling ill or well. We are never satisfied by merely ascertaining
the fact that we feel thus or thus: we admit this

INFO: Selected album #3: Schubert: Piano Trio Nr. 2, D.929: Trio-Sonate, D. 28 by Beaux Arts Trio



ALBUM SELECTION
REASONING: The excerpt by Nietzsche presents a philosophical exploration of human consciousness and the tendency to attribute causes to our feelings. An album that complements this theme would likely feature complex, atmospheric soundscapes that reflect the introspective nature of the text. Among the options, "Schubert: Piano Trio Nr. 2, D.929: Trio-Sonate, D. 28" by Beaux Arts Trio (1993) stands out for its soothing, contemplative quality, which aligns with Nietzsche's emphasis on introspection and philosophical inquiry.

VERDICT: 3


# Aggregate Album Data

Fetch the selected album's track list and album art. Then, get visual model to describe the album art if applicable.

In [ ]:
from daily_greeting import get_album_details
import base64

album_details = get_album_details(album['id'])
album['songs'] = album_details.get('songs')

with open("./tmp/album_art.jpg", "wb") as f:
    f.write(base64.b64decode(album_details['coverart']))

if not album_details['coverart']:
    logging.warning("No cover art found for selected album.")
    album['coverart'] = None
else:
    default_prompt = f"""Determine whether this image matches the default album cover image for Navidrome, a blue vinyl record on a blank background with the word "Navidrome" on it.

Respond with the following exact format strictly:
DESCRIPTION: One sentence description of the image.
REASONING: One sentence reasoning about whether the cover art matches not.
VERDICT: YES if it matches NO if it does not"""
    
    print("\n" + "="*50)
    print("COVER ART DEFAULT CHECK PROMPT")
    print("="*50)
    print(default_prompt)

    response = send_ollama_image_request(default_prompt, album_details['coverart'], keep_alive=5)
    print("\n" + "="*50)
    print("COVER ART DEFAULT CHECK RESPONSE")
    print("="*50)
    print(response)

    if response is None:
        logging.error("Failed to get response for cover art default check.")
        album['coverart'] = None
    elif "VERDICT: YES" in response.upper():
        logging.info("Cover art is default, discarding.")
        album['coverart'] = None
    else:
        logging.info("Cover art is valid, analyzing.")

        art_prompt = f"""Provide a detailed desrtiption of the provided album cover art, including colors, objects, and style. Use two or three sentences."""

        print("\n" + "="*50)
        print("COVER ART ANALYSIS PROMPT")
        print("="*50)
        print(art_prompt)

        analysis = send_ollama_image_request(art_prompt, album_details['coverart'])

        if analysis is None:
            logging.error("Failed to get response for cover art analysis.")
            album['coverart'] = None
        else:
            print("\n" + "="*50)
            print("COVER ART ANALYSIS")
            print("="*50)
            print(analysis)
            album['coverart'] = analysis

logging.info("Album Details:")
logging.info(json.dumps(album, indent=2))

INFO: Fetching details for album ID 6c7TZV1qbzEQKxk1qKwgLg from Navidrome server...
INFO: Sending image request to Ollama model gemma3:4b...



COVER ART DEFAULT CHECK PROMPT
Determine whether this image matches the default album cover image for Navidrome, a blue vinyl record on a blank background with the word "Navidrome" on it.

Respond with the following exact format strictly:
DESCRIPTION: One sentence description of the image.
REASONING: One sentence reasoning about whether the cover art matches not.
VERDICT: YES if it natches NO if it does not


INFO: Cover art is valid, analyzing.
INFO: Sending image request to Ollama model gemma3:4b...



COVER ART DEFAULT CHECK RESPONSE
DESCRIPTION: The image shows a close-up of a man with glasses.
REASONING: The image depicts a man's face, which is not the typical Navidrome album cover design.
VERDICT: NO

COVER ART ANALYSIS PROMPT
Provide a detailed desrtiption of the provided album cover art, including colors, objects, and style. Use two or three sentences.


INFO: Album Details:
INFO: {
  "id": "6c7TZV1qbzEQKxk1qKwgLg",
  "name": "Schubert: Piano Trio Nr. 2, D.929: Trio-Sonate, D. 28",
  "artist": "Beaux Arts Trio",
  "year": 1993,
  "genres": [
    "Chamber Music"
  ],
  "songs": [
    "Piano Trio No. 2 in E flat major, D. 929 (Op. 100): Allegro",
    "Piano Trio No. 2 in E flat major, D. 929 (Op. 100): Andante con moto",
    "Piano Trio No. 2 in E flat major, D. 929 (Op. 100): Scherzo (Allegro moderato)",
    "Piano Trio No. 2 in E flat major, D. 929 (Op. 100): Allegro moderato",
    "Piano Trio in B flat major (\"Sonatensatz\"), D. 28"
  ],
  "coverart": "Here's a detailed description of the album cover art:\n\nThe image presents a close-up, slightly blurred photograph of a man with eyeglasses, set against a dark, indistinct background. The primary color is a deep, muted gray, creating a somber and contemplative mood, while the man's features - the sharp angle of his nose and the seriousness of his expression - are sharply defined. The 


COVER ART ANALYSIS
Here's a detailed description of the album cover art:

The image presents a close-up, slightly blurred photograph of a man with eyeglasses, set against a dark, indistinct background. The primary color is a deep, muted gray, creating a somber and contemplative mood, while the man's features - the sharp angle of his nose and the seriousness of his expression - are sharply defined. The overall style leans toward a realistic portrait with a touch of ambiguity due to the shallow depth of field.
